## AndLink and EvaluationLink examples

In [17]:
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import bindlink
from opencog.bindlink import execute_atom

create atomspace and set it as default

In [18]:
atomspace = AtomSpace()
initialize_opencog(atomspace)

### knowledge base

Color and size are attributes

In [19]:

InheritanceLink(ConceptNode("size"), ConceptNode("attribute"))
InheritanceLink(ConceptNode("color"), ConceptNode("attribute"))



(InheritanceLink
  (ConceptNode "color") ; [4437725137865661335][6]
  (ConceptNode "attribute") ; [8286119068776667766][6]
) ; [15737919214045963885][6]

different types of color and size

In [20]:
InheritanceLink(ConceptNode("small"), ConceptNode("size"))
InheritanceLink(ConceptNode("big"), ConceptNode("size"))
InheritanceLink(ConceptNode("green"), ConceptNode("color"))
InheritanceLink(ConceptNode("red"), ConceptNode("color"))
InheritanceLink(ConceptNode("black"), ConceptNode("color"))

(InheritanceLink
  (ConceptNode "black") ; [3641150941895841241][6]
  (ConceptNode "color") ; [4437725137865661335][6]
) ; [17348854327767872816][6]

items which have some attributes defined

In [21]:
InheritanceLink(ConceptNode("item-0"), ConceptNode("small")).tv = TruthValue(0.4, 0.9)
InheritanceLink(ConceptNode("item-0"), ConceptNode("green")).tv = TruthValue(0.3, 0.9)
InheritanceLink(ConceptNode("item-2"), ConceptNode("big")).tv = TruthValue(0.4, 0.9)
InheritanceLink(ConceptNode("item-2"), ConceptNode("red")).tv = TruthValue(0.4, 0.9)
InheritanceLink(ConceptNode("item-1"), ConceptNode("red")).tv = TruthValue(0.2, 0.9)
InheritanceLink(ConceptNode("item-1"), ConceptNode("small")).tv = TruthValue(0.8, 0.9)
InheritanceLink(ConceptNode("item-3"), ConceptNode("black")).tv = TruthValue(0.14, 0.9)
InheritanceLink(ConceptNode("item-3"), ConceptNode("small")).tv = TruthValue(0.38, 0.9)

all items are items

In [22]:
InheritanceLink(ConceptNode("item-0"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-0"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-2"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-2"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-1"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-1"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-3"), ConceptNode("item"))
InheritanceLink(ConceptNode("item-3"), ConceptNode("item"))

(InheritanceLink
  (ConceptNode "item-3") ; [3557986846555413044][6]
  (ConceptNode "item") ; [6497446589895348736][6]
) ; [12252957330622942996][6]

let's select all items and attributes, filtering by some threshold


first define function and EvaluationLink

In [23]:
threshold = 0.3

def apply_threshold(atom):
    if atom.tv.mean < threshold:
        return TruthValue(0, 1)
    return TruthValue(1, 1)

EvaluationLink will accept link    X <- Z  
and return flag if it "matches"

In [24]:
inherits_property = InheritanceLink(VariableNode("X"), VariableNode("Z"))
# all Variables(X and Z) used in EvaluationLink should be grounded somewhere else first!
# eval link which provides filtering
eval_threshold = EvaluationLink(
                    GroundedPredicateNode("py: apply_threshold"),
                    ListLink(inherits_property))

### now build the BindLink

Constrain that Variable "X" inherits from "item"

In [25]:
select_items = InheritanceLink(VariableNode("X"), ConceptNode("item"))

In [26]:
# using NotLink
Z_is_not_item = NotLink(IdenticalLink(VariableNode("Z"), ConceptNode("item")))

and_link = AndLink(eval_threshold, select_items, inherits_property, Z_is_not_item)
bindlink_filter = BindLink(and_link,
                           inherits_property)

print(bindlink(atomspace, bindlink_filter))



(SetLink
  (InheritanceLink (stv 0.400000 0.900000)
    (ConceptNode "item-0")
    (ConceptNode "small")
  )
  (InheritanceLink (stv 0.800000 0.900000)
    (ConceptNode "item-1")
    (ConceptNode "small")
  )
  (InheritanceLink (stv 0.300000 0.900000)
    (ConceptNode "item-0")
    (ConceptNode "green")
  )
  (InheritanceLink (stv 0.400000 0.900000)
    (ConceptNode "item-2")
    (ConceptNode "red")
  )
  (InheritanceLink (stv 0.380000 0.900000)
    (ConceptNode "item-3")
    (ConceptNode "small")
  )
  (InheritanceLink (stv 0.400000 0.900000)
    (ConceptNode "item-2")
    (ConceptNode "big")
  )
)



## Exercise: remove Z_is_not_item from condition